# <span style="color:hotpink"> PySpark labo </span>

Tijdens dit deel van het labo zal je de PySpark basics leren en hoe ze toe te passen voor machine learning.  
De dataset waarmee gewerkt zal worden is data over huizen in king county en op het einde zal je de prijs van een huis proberen te voorspellen aan de hand van lineaire regressie.  
De kolommen die aanwezig zijn in de dataset en hun beschrijving zijn:  
`price`: prediction target  
`bedrooms`: number of bedrooms/house  
`bathrooms`: number of bathrooms/house  
`sqft_living`: square footage of the home  
`sqft_lot`: square footage of the lot  
`floors`: Total floors(levels) in a house  
`waterfront`: House which has a view to a waterfront  
`view`: Has been viewed  
`condition`: How good the condition overall is  
`grade`: overall grade given to the housing unit, based on King County grading system  
`sqft_above`: square footage of house apart from basement  
`sqft_basement`: square footage of the basement  
`yr_built`: built year  
`yr_renovated`: Year when house was renovated  
`zipcode`: zip  
`sqft_living15`: Living room area in 2015(implies some renovations) This might or might not have affected the lotsize area
`sqft_lot15`: LotSize area in 2015(implies some renovation) 

De te beantwoorden vragen/opdrachten staan altijd in italics. 

In [2]:
import findspark
findspark.init('/home/marie/spark')
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

## <span style="color:hotpink"> SparkSession, data inlezen en data weergeven</span>
Een SparkSession voorziet een single point of entry om te interageren met de onderliggende Spark functionaliteit en laat de gebruiker toe om te programmeren met de DataFrame API.  
De te gebruiken functie om een sparksessie te bouwen is:  
`spark = SparkSession.builder.appName('name').getOrCreate()`  
*Bouw nu zelf een SparkSession met de naam labo.*

De data kan ingelezen worden met de functie: `spark.read.csv()`  
*Laad de data kc_house_data.csv in en zet de parameters `inferSchema` en `header` op True*

De functie `df.printSchema()` toont alle kolommen en het type waarden dat hierin aanwezig zijn.  

*Wat is het type waarden aanwezig in de kolom sqft_basement?*

Om een spark DataFrame (of spark column) weer te geven moet je de functie `df.show()` aanroepen. Dus telkens een functie een Spark Dataframe returned moet je `.show()` gebruiken om het resultaat te kunnen zien.  
*Bekijk nu zelf de data.*

Om de namen van de kolom weer te geven als lijst kan je de functie:  
`df.columns` gebruiken

# <span style="color:hotpink"> selecteren van kolommen </span>
Je kan een kolom selecteren aan de hand van de methode:  
`df["column_name"]`  
Selecteer de kolom condition

Het verkregen type is een Column, maar met de functie `df.select('column_name')` returened een spark DataFrame, die veelzijdiger is dan het type Column.  
*Selecteer opnieuw de kolom condition maar nu als dataframe.*

Om de inhoud van de geselecteerde kolom weer te geven moet je de methode `.show` aanroepen.

`df.head(n)` geeft de eerste n rijen terug als lijst.  
Print de eerste 5 rijen

Je kan met `data.head()` ook een specifieke rij of een bepaalde waarde selecteren met behulp van:  
`data.head(n)[row_number][column_number]`  
*1) Selecteer de 4de rij*  
*2) Selecteer prijs op de 3de rij*

*Is dit een action of transformation?*

# <span style="color:hotpink"> Creating a new column and dropping columns </span>

Je kan een nieuwe kolom aanmaken met de functie: `data.withColumn('new_column_name',data["column"])`  
data["column"] kan ook bewerkt worden bv: `data.withColumn('new_column_name',data["column"]/2)`  
Een kolom kan verwijdert worden met `data.drop('column')`  
_Verander nu alle sqft kolommen naar m2 (*0.0929) en verwijder de sqft kolommen._

*Is dit een action of transformation?*

# <span style="color:hotpink"> Filtering and grouping data </span>

## <span style="color:hotpink"> Filtering Data </span>

Voor het werken met big data is het belangrijk dat je snel je data kan filteren gebaseerd op bepaalde condities.  

Je kunt de dataset filteren met behulp van de methode:  
`df.filter(df["column_name"] < condition)`  
Je kan ook filteren voor meerdere condities met behulp van `|` of `&`

*Filter de dataset voor huizen die gebouwd zijn na het jaar 2000, sla dit op onder de variabele `data_2000`*

Met de `.count()` methode kan je het aantal gefilterde huizen opvragen.  
*Hoeveel huizen bevat `dataset_2000` met minimum 2 slaapkamers en die gelegen zijn aan het waterfront?*

## <span style="color:hotpink"> GroupBy and Aggregate Functions </span>

De `groupBy('column_name')` functie laat je rijen groeperen gebaseerd op een bepaalde kolom, bijvoorbeeld je kan huizen groeperen volgens bouwjaar.  
Eenmaal je de rijen gegroepeerd hebt, kan je meerdere rijen van data aggregeren tot een output, bijvoorbeeld door het nemen van de som van alle inputrijen of de minimum waarde.  
*Gebruik de groupBy functie op de 'condition' kolom en vraag de minimum waarde op met de .min() functie.*

*Groepeer de rijen volgens de `waterfront` kolom en aggregeer volgens het gemiddelde.*

Niet alle methoden hebben nood aan een groupby call. In plaats darvan kan je de algemene agg() methode oproepen. 
Dit kan alle rijen in de dataframe aggregeren in een kolom.  
`df.agg({'column_name':'operation'})` de operation kan mean,min,max,count en sum zijn.

*Gebruik de algemene agg() methode om de gemiddelde prijs te vinden*

Deze functie geeft echter een DataFrame terug, wat niet zo handig is bv. als je het gemiddelde nodig hebt om berekeningen uit te voeren. Om enkel het getal terug te krijgen moet je het getal als volgt uit de spark DataFrame halen:  
`[row_number][column_number]`  
*Zorg ervoor dat de algemene agg() methode het gemiddelde als getal terug geeft.*

*Groepeer eerst alle rijen volgens `yr_renovated` en sla dit op als de variabele `grouped`. 
Gebruik vervolgens de algemene agg() methode om de maximale prijs te vinden per jaar*

# <span style="color:hotpink"> Machine learning with PySpark </span>

Om machine learning te kunnen toepassen met MlLib verwacht spark een format dat 2 kolommen bevat met de namen: "label" en "features".  
De label kolom moet een numerische label bevatten, dit kan een numerische waarde zijn voor regressie of classificatie.  
De feature kolom moet een vector van alle features bevatten.  
In deze sectie zal je een dataframe omzetten naar het verwachte format en zal je de huisprijzen proberen te voorspellen met lineaire regressie.

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# <span style="color:hotpink"> Prepping the data </span>

## <span style="color:hotpink"> Prepping the data </span>
Om de data correct voor te bereiden moeten alle features in een kolom onder de vorm van een vector aanwezig zijn.
Dit kan bereikt worden met de functies: `assembler = VectorAssembler(inputCols=[list_of_column_names],outputCol='features')`  
`new_dataframe = assembler.transform(df)`  
*Maak een lijst met alle kolomnamen die gebruikt kunnen worden als features en transformeer ze met de VectorAssembler.*

## <span style="color:hotpink"> Splitsen in train en test data </span>

Om de data op te delen in train en test data kun je de volgende functie gebruiken:  
`train_data,test_data = df.randomSplit([float1,float2])`  
*Splits nu de data op waarbij 80% van de data train data is.*

## <span style="color:hotpink"> Runnen algoritme </span>

We zullen in dit labo werken met lineaire regressie. De te gebruiken functies zijn:  
`lr = LinearRegression(labelCol = 'column_name_target', featuresCol = 'column_name_features, regParam = float)`  
`model = lr.fit(train_data)`  
de parameter `regParam` == lambda.

*Extra oefening: maak zelf een min-max scaler met de geziene functies en pas deze toe op de dataset. Resulteert dit in een betere MSE?*